In [ ]:
%matplotlib inline
#
from lrgsglib.shared import *
from lrgsglib.config.const import *
from lrgsglib.config.plotlib import inset_axes, restr_twilight
from lrgsglib.config.utils import read_files_to_2d_array
from lrgsglib.core import move_to_rootf, Lattice2D
from matplotlib.ticker import ScalarFormatter
from scipy.signal import find_peaks
#
move_to_rootf()
plt.style.use("ipynb/nb_plotsheet.mplstyle")
#
suppinfPath = "data/plot/paper/suppinf/"
datasubPathStr = ["data_pablo_tri-hex", "data_pablo_2-3-4-D"]
figsubpathstr = ['phtra']
paths = {str: f"{suppinfPath}{str}/" for str in figsubpathstr}
for path in paths.values(): os.makedirs(path, exist_ok=True)
#
axvline_kwarg = dict(ls=':', color='k')
xlabelinset_kwargs = dict(labelpad=0, fontsize=18)
ylabelinset_kwargs = dict(rotation=90, labelpad=10, va="center",fontsize=18)
axins_tickparams_kwargs = dict(axis='both', which='major', size=7, width=1.25, labelsize=16)
axlegend_kwargs = dict(ncol=1, loc='lower left', fontsize=18)
kwargs_lines = dict(lw=5, marker='')

In [2]:
def process_plot(fig, ax, sides, geo, column, pathfname, legend_pos=None, 
                 ylabel='', text_content='', text_pos=None,
                 transform=lambda x, L: x, transvline=False):
    colors = restr_twilight(np.linspace(0., 1., len(sides)))
    plt.rcParams["axes.prop_cycle"] = plt.cycler(color=colors)
    for cc, side in enumerate(sides):
        if geo in ['tri', 'hex']:
            l = Lattice2D(side, geo)
            N = l.N
        elif geo == '4D':
            N = side**4
        fname = f"{pathfname}{side}"
        data = np.loadtxt(fname, delimiter=',')
        y_data = gaussian_filter(data[:, column], sigma=1)
        p = data[:, 9]
        if transform:
            y_data = transform(y_data, side)
        ax.plot(p, y_data, label=fr'${N}$', zorder=1/(cc+1), 
                **kwargs_lines)
    ax.set_xlabel(r'$p$')
    ax.set_ylabel(ylabel)
    if text_pos:
        ax.text(*text_pos, text_content, transform=ax.transAxes, fontsize=40, 
                fontweight='bold', color='black')
    if transvline:
        peaks, _ = find_peaks(gaussian_filter(data[:, column], sigma=3))
        ax.axvline(p[peaks[0]], ls='--', c='k')
    ax.legend(fontsize=28, loc=legend_pos if legend_pos else 'upper right')
    fig.tight_layout()

## Phase Transitions

### Hex: Size of the giant Cluster $P_{\infty}$ and Fluctuations $\chi$

In [ ]:
sides = [16, 32, 64, 128, 256]
geo = 'hex'
pathfname = os.path.join(suppinfPath, datasubPathStr[0], "OP_Hex_L")

In [ ]:
fig_1, ax_1 = plt.subplots(figsize=(11, 7))
fig_2, ax_2 = plt.subplots(figsize=(11, 7))

process_plot(fig_1, ax_1, sides, geo, 0, pathfname, 
             ylabel=r'$P_{\infty}$')#, text_pos=(0.03, 0.06), text_content=r'\textbf{(a)}')
process_plot(fig_2, ax_2, sides, geo, 1, pathfname, 
             transform=lambda x, L: x*L, ylabel=r'$\chi$', 
             legend_pos=(0.6, 0.35), transvline=True)# text_pos=(0.88, 0.88), text_content=r'\textbf{(b)}')

# Save figures
fig_1.savefig(f"{paths['phtra']}suppli_S_{geo}.pdf", 
              bbox_inches='tight', transparent=True)
fig_2.savefig(f"{paths['phtra']}suppli_fS_{geo}.pdf", 
              bbox_inches='tight', transparent=True)

plt.close('all')

### Tri: Size of the giant Cluster $P_{\infty}$ and Fluctuations $\chi$

In [ ]:
sides = [16, 32, 64, 96]
geo = 'tri'
pathfname = os.path.join(suppinfPath, datasubPathStr[0], "OP_Tri_L")

In [ ]:
fig_1, ax_1 = plt.subplots(figsize=(11, 7))
fig_2, ax_2 = plt.subplots(figsize=(11, 7))

process_plot(fig_1, ax_1, sides, geo, 0, pathfname, 
             ylabel=r'$P_{\infty}$')#, text_pos=(0.03, 0.06), text_content=r'\textbf{(a)}')
process_plot(fig_2, ax_2, sides, geo, 1, pathfname, 
             transform=lambda x, L: x*L, ylabel=r'$\chi$', 
             legend_pos='upper right', transvline=True)# text_pos=(0.88, 0.88), text_content=r'\textbf{(b)}')

# Save figures
fig_1.savefig(f"{paths['phtra']}suppli_S_{geo}.pdf", 
              bbox_inches='tight', transparent=True)
fig_2.savefig(f"{paths['phtra']}suppli_fS_{geo}.pdf", 
              bbox_inches='tight', transparent=True)

plt.close('all')

### 4D: Size of the giant Cluster $P_{\infty}$ and Fluctuations $\chi$

In [ ]:
sides = [7, 8, 9, 10]
geo = '4D'
pathfname = os.path.join(suppinfPath, datasubPathStr[1], "OP_Squared4D_L")

In [ ]:
fig_1, ax_1 = plt.subplots(figsize=(11, 7))
fig_2, ax_2 = plt.subplots(figsize=(11, 7))

process_plot(fig_1, ax_1, sides, geo, 0, pathfname, legend_pos='lower left',
             ylabel=r'$P_{\infty}$')#, text_pos=(0.03, 0.06), text_content=r'\textbf{(a)}')
process_plot(fig_2, ax_2, sides, geo, 1, pathfname, legend_pos='upper left',
             transform=lambda x, L: x*L*L, ylabel=r'$\chi$', 
             transvline=True)# text_pos=(0.88, 0.88), text_content=r'\textbf{(b)}')

# Save figures
fig_1.savefig(f"{paths['phtra']}suppli_S_{geo}.pdf", 
              bbox_inches='tight', transparent=True)
fig_2.savefig(f"{paths['phtra']}suppli_fS_{geo}.pdf", 
              bbox_inches='tight', transparent=True)

plt.close('all')

### Tri ZERR: Size of the giant Cluster $P_{\infty}$ and Fluctuations $\chi$

In [7]:
sides = [16, 32, 64, 96, 128]
geo = 'tri'
pathfname = os.path.join(suppinfPath, datasubPathStr[0], "OP_Zerr_Tri_L")

In [8]:
fig_1, ax_1 = plt.subplots(figsize=(11, 7))
fig_2, ax_2 = plt.subplots(figsize=(11, 7))

process_plot(fig_1, ax_1, sides, geo, 0, pathfname, 
             ylabel=r'$P_{\infty}$')#, text_pos=(0.03, 0.06), text_content=r'\textbf{(a)}')
process_plot(fig_2, ax_2, sides, geo, 1, pathfname, 
             transform=lambda x, L: x*L, ylabel=r'$\chi$', 
             legend_pos='upper right', transvline=True)# text_pos=(0.88, 0.88), text_content=r'\textbf{(b)}')

# Save figures
fig_1.savefig(f"{paths['phtra']}suppli_S_{geo}Zerr.pdf", 
              bbox_inches='tight', transparent=True)
fig_2.savefig(f"{paths['phtra']}suppli_fS_{geo}Zerr.pdf", 
              bbox_inches='tight', transparent=True)

plt.close('all')

### Hex XERR: Size of the giant Cluster $P_{\infty}$ and Fluctuations $\chi$

In [13]:
sides = [16, 32, 64, 128]
geo = 'hex'
pathfname = os.path.join(suppinfPath, datasubPathStr[0], "OP_Xerr_Hex_L")

In [14]:
fig_1, ax_1 = plt.subplots(figsize=(11, 7))
fig_2, ax_2 = plt.subplots(figsize=(11, 7))

process_plot(fig_1, ax_1, sides, geo, 0, pathfname, 
             ylabel=r'$P_{\infty}$')#, text_pos=(0.03, 0.06), text_content=r'\textbf{(a)}')
process_plot(fig_2, ax_2, sides, geo, 1, pathfname, 
             transform=lambda x, L: x*L, ylabel=r'$\chi$', 
             legend_pos=(0.6, 0.35), transvline=False)# text_pos=(0.88, 0.88), text_content=r'\textbf{(b)}')

# Save figures
fig_1.savefig(f"{paths['phtra']}suppli_S_{geo}Xerr.pdf", 
              bbox_inches='tight', transparent=True)
fig_2.savefig(f"{paths['phtra']}suppli_fS_{geo}Xerr.pdf", 
              bbox_inches='tight', transparent=True)

plt.close('all')

## Phase transition ZERR/XERR

In [ ]:
errtype = 'Xerr'

### Squared

In [ ]:
sides = [16, 32, 64, 96]
geo = 'squared'
#
colors = restr_twilight(np.linspace(0.0, 1, len(sides)))
plt.rcParams["axes.prop_cycle"] = plt.cycler(color=colors)

In [ ]:
fig, ax = plt.subplots(figsize=(22, 8), ncols=2)

for cc, side in enumerate(sides):
    try:
        l = Lattice2D(side, geo, init_nw_dict=False, with_positions=False)
        fname = f"data/plot/paper/suppinf/data_pablo_tri-hex/OP_{errtype}_Squared_L{side}"
        data = np.loadtxt(fname, delimiter=',')
        p = data[:, 9]
        p_c = data[np.argmax(data[:, 1]), 9]
        S = gaussian_filter(data[:, 0], sigma=1)
        L = side
        # plt.plot(p, S, label=f'L={side}', lw=2)
        ax[0].plot(p, S, label=fr'$N={l.N}$', zorder=1/(cc+1), **kwargs_lines)
    except FileNotFoundError:
        print(f"File not found for L={side}")
#
# ax[0].axhline(0.5, ls='--', c='k')
#
ax[0].set_xlabel(r'$p$')
ax[0].set_ylabel(r'$P_{\infty}$')
#
ax[0].text(0.03, 0.06, r'\textbf{(a)}', transform=ax[0].transAxes, fontsize=40,
                 fontweight='bold', color='black')
ax[0].legend(fontsize=28)

plt.rcParams["axes.prop_cycle"] = plt.cycler(color=colors)

for cc,side in enumerate(sides):
    try:
        l = Lattice2D(side, geo, init_nw_dict=False, with_positions=False)
        fname = f"data/plot/paper/suppinf/data_pablo_tri-hex/OP_{errtype}_Squared_L{side}"
        data = np.loadtxt(fname, delimiter=',')
        p = data[:, 9]
        p_c = data[np.argmax(data[:, 1]), 9]
        fS = gaussian_filter(data[:, 1], sigma=1)
        L = side
        # plt.plot(p, S, label=f'L={side}', lw=2)
        ax[1].plot(p, fS*L, label=fr'$N={l.N}$', zorder=1/(cc+1), **kwargs_lines)
    except FileNotFoundError:
        print(f"File not found for L={side}")
#
ax[1].axvline(0.10, ls='--', c='k')
#
# ax[1].set_yticks([])
formatter = ScalarFormatter(useMathText=True)  # Enable the use of math text in scientific notation
formatter.set_scientific(True)  # Enable scientific notation
formatter.set_powerlimits((-1,1))  # Set limits for switching to scientific notation
ax[1].yaxis.set_major_formatter(formatter)

ax[1].set_xlabel(r'$p$')
ax[1].set_ylabel(r'$\chi$')
#
ax[1].text(0.88, 0.88, r'\textbf{(b)}', transform=ax[1].transAxes, fontsize=40,
                 fontweight='bold', color='black', zorder=100000)
ax[1].legend(fontsize=26, bbox_to_anchor=(.6, .35))
fig.tight_layout()
fig.subplots_adjust(wspace=.5)  # 'wspace' controls the width of the space between subplots
#
# fig.savefig(f"{paths['phtra']}suppli_both_{errtype}_{geo}.pdf", bbox_inches='tight',
#             transparent=True)
# plt.close('all')

### Triangular

In [ ]:
sides = [16, 32, 64, 96]
geo = 'tri'
#
colors = restr_twilight(np.linspace(0.0, 1, len(sides)))
# Set these colors as the default color cycle
plt.rcParams["axes.prop_cycle"] = plt.cycler(color=colors)

In [ ]:
fig, ax = plt.subplots(figsize=(22, 8), ncols=2)

for cc, side in enumerate(sides):
    try:
        l = Lattice2D(side, geo, init_nw_dict=False, with_positions=False)
        fname = f"data/plot/paper/suppinf/data_pablo_tri-hex/OP_{errtype}_Tri_L{side}"
        data = np.loadtxt(fname, delimiter=',')
        p = data[:, 9]
        p_c = data[np.argmax(data[:, 1]), 9]
        S = gaussian_filter(data[:, 0], sigma=1)
        L = side
        # plt.plot(p, S, label=f'L={side}', lw=2)
        ax[0].plot(p, S, label=fr'$N={l.N}$', zorder=1/(cc+1), **kwargs_lines)
    except FileNotFoundError:
        print(f"File not found for L={side}")
#
# ax[0].axhline(0.5, ls='--', c='k')
#
ax[0].set_xlabel(r'$p$')
ax[0].set_ylabel(r'$P_{\infty}$')
#
ax[0].text(0.03, 0.06, r'\textbf{(a)}', transform=ax[0].transAxes, fontsize=40,
                 fontweight='bold', color='black')
ax[0].legend(fontsize=28)

plt.rcParams["axes.prop_cycle"] = plt.cycler(color=colors)

for cc,side in enumerate(sides):
    try:
        l = Lattice2D(side, geo, init_nw_dict=False, with_positions=False)
        fname = f"data/plot/paper/suppinf/data_pablo_tri-hex/OP_{errtype}_Tri_L{side}"
        data = np.loadtxt(fname, delimiter=',')
        p = data[:, 9]
        p_c = data[np.argmax(data[:, 1]), 9]
        fS = gaussian_filter(data[:, 1], sigma=1)
        L = side
        # plt.plot(p, S, label=f'L={side}', lw=2)
        ax[1].plot(p, fS*L, label=fr'$N={l.N}$', zorder=1/(cc+1), **kwargs_lines)
    except FileNotFoundError:
        print(f"File not found for L={side}")
#
ax[1].axvline(0.14, ls='--', c='k')
#
# ax[1].set_yticks([])
formatter = ScalarFormatter(useMathText=True)  # Enable the use of math text in scientific notation
formatter.set_scientific(True)  # Enable scientific notation
formatter.set_powerlimits((-1,1))  # Set limits for switching to scientific notation
ax[1].yaxis.set_major_formatter(formatter)

ax[1].set_xlabel(r'$p$')
ax[1].set_ylabel(r'$\chi$')
#
ax[1].text(0.88, 0.88, r'\textbf{(b)}', transform=ax[1].transAxes, fontsize=40,
                 fontweight='bold', color='black', zorder=100000)
ax[1].legend(fontsize=26, bbox_to_anchor=(.6, .45))
fig.tight_layout()
fig.subplots_adjust(wspace=.5)  # 'wspace' controls the width of the space between subplots
#
fig.savefig(f"{paths['phtra']}suppli_both_{errtype}_{geo}.pdf", bbox_inches='tight',
            transparent=True)
# plt.close('all')